### Memory

In [1]:
from langchain_core.chat_history import BaseChatMessageHistory

# 채팅 기록을 저장할 클래스 정의
# BaseChatMessageHistory를 상속받는다.
class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []  # 메세지들을 저장할 빈 리스트 생성

    def add_messages(self, messages):
        # 메세지 리스트가 들어오면 기존 리스트에 추가
        self.messages.extend(messages)

    def clear(self):
        # 기록 초기화
        self.messages = []

# 세션(사용자)별로 기록을 관리할 딕셔너리 생성
# store = {'user1' : History객체, 'user2' : History객체, ...}
store = {}

# 세션 ID를 받아 해당 세션의 History 객체를 반환하는 함수
def get_by_session_id(session_id):
    if session_id not in store:
        # 새로운 InMemoryHistory 객체를 만들어서 store에 저장
        store[session_id] = InMemoryHistory()
    # 해당하는 session_id 기록 객체를 반환
    return store[session_id]

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 프롬프트 템플릿 생성
prompts = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 도우미야'), # 시스템 메세지 (페르소나 설정)
    MessagesPlaceholder(variable_name="history"), # 대화 기록이 주입될 위치
    ('human', '{query}') # 사용자 질문
])

# 모델 생성
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.8)

# 기본 체인 생성
chain = prompts | model

#  메모리 기능 장착한 체인 생성(RunnableWithMessageHistory)
#  기존 chain을 감싸서, 대화 기록을 자동으로 읽고 쓰는 기능 추가
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id, # 세션 ID로 기록을 찾아오는 함수 연결
    input_messages_key='query',  # 사용자 입력이 들어오는 변수명
    history_messages_key='history'  # 프롬프트에서 기록이 들어갈 변수명 (Placeholder 이름과 일치해야 함)
)

In [6]:
response = chain_with_history.invoke(
    {'skill' : 'math', 'query' : '철수는 강아지를 세마리 키우고 있습니다.'},
    config={'configurable':{'session_id': 'abc'}}   # session_id를 'abc'로 설정
)

print(response.content)

좋아요! 그렇다면 강아지와 관련된 문제를 하나 만들어보겠습니다.

**문제:**
철수가 키우는 세 강아지의 몸무게는 각각 5kg, 7kg, 9kg이다. 이 세 강아지의 몸무게를 모두 더한 후, 평균 몸무게를 구하시오.

**풀이:**
세 강아지의 몸무게를 각각 a, b, c로 표시하면 a=5kg, b=7kg, c=9kg이다.
몸무게의 합은 S = a + b + c = 5 + 7 + 9 = 21kg 이다.
평균 몸무게는 M = (a + b + c) / 3 = 21 / 3 = 7kg 이다.

따라서, 철수가 키우는 세 강아지의 몸무게의 합은 21kg이며, 평균 몸무게는 7kg입니다.

이런 문제로 재미있게 수학을 공부해보실 수 있습니다. 만약 다른 유형의 문제를 원하시거나 더 진행하고 싶은 문제가 있다면 알려주세요!


In [7]:
response2 = chain_with_history.invoke(
    {'skill':'math', 'query': '영희가 고양이를 두마리 키우고 있습니다.'},
    config={'configurable': {'session_id' : 'abc'}}
)

print(response2.content)

좋아요, 그럼 두마리의 고양이와 관련된 문제를 만들어보겠습니다.

**문제:**
영희가 키우는 두 마리의 고양이 중 한 마리의 몸무게는 3kg이고, 다른 한 마리의 몸무게는 4kg이다. 이 두 고양이의 몸무게를 모두 더한 후, 평균 몸무게를 구하시오.

**풀이:**
두 고양이의 몸무게를 각각 x, y로 표시하면 x=3kg, y=4kg이다.
몸무게의 합은 S = x + y = 3 + 4 = 7kg 이다.
평균 몸무게는 M = (x + y) / 2 = 7 / 2 = 3.5kg 이다.

따라서, 영희가 키우는 두 고양이의 몸무게의 합은 7kg이며, 평균 몸무게는 3.5kg입니다.

이렇게 간단한 문제를 풀어보며 수학적 사고력을 향상시킬 수 있습니다. 더 풀고 싶은 문제가 있거나 특정 유형의 문제를 원하시면 알려주세요!


In [ ]:
# 문맥을 파악해야만 풀 수 있는 문제
response3 = chain_with_history.invoke(
    {'skill':'math', 'query': '철수랑 영희랑 합해서 몇 마리의 동물을 키우고 있나요??'},
    config={'configurable': {'session_id' : 'abc'}}
)

print(response3.content)

철수는 강아지 2마리와 고양이 1마리를 키우고 있고, 영희는 토끼 3마리를 키우고 있습니다. 따라서 철수와 영희가 합쳐서 키우고 있는 동물은 6마리입니다.


In [11]:
# 저장소 확인
print(store['abc'])

Human: 철수는 강아지를 세마리 키우고 있습니다.
AI: 멋진 설정이에요! 어떤 수학 문제를 풀고 싶은지 알려주시면 그에 맞춰 도와드릴게요. 먼저 이 상황으로 바로 쓸 수 있는 예시 문제들 몇 가지를 드리겠습니다.

예시 문제들
- 합과 평균: 강아지들의 나이를 a, b, c라고 할 때, 합 S = a + b + c, 평균 M = (a + b + c) / 3. 예: a=2, b=4, c=5면 S=11, M=11/3.
- 산책 동행의 경우의 수: 세 강아지 중 두 마리를 골라 함께 산책시키는 방법의 수는 C(3,2) = 3개.
- 목걸이 색상 배치: 서로 다른 색깔의 목걸이 3개를 세 강아지에게 하나씩 줄 때의 배치 수는 3! = 6개.
- 간식 분배(최소 1개씩): 간식 10개를 세 강아지에게 최소 1개씩 주려면 각 강아지가 받는 간식의 수 x1, x2, x3가 xi ≥ 1이고 x1+x2+x3 = 10이 됩니다. 경우의 수는 스타-앤-바스: C(9,2) = 36가지.
- 비율 분배 예시: 세 강아지의 체중이 2kg, 3kg, 5kg일 때 같은 비율로 간식의 양을 나누면 각 강아지가 받게 되는 비율은 2:3:5이고, 전체 양이 10단위일 때 각각 2, 3, 5가 됩니다(설정에 따라 단위 조정 가능).
- 확률 개념 예시: 세 강아지 중 한 마리가 특정 간식을 뽑을 확률은 각각 1/3으로 동일하다고 가정하면, 특정 강아지가 뽑힐 확률은 1/3.

원하시는 방향이나 난이도(예: 초등 수준 덧셈/나눗셈, 확률, 조합, 방정식 등)를 알려주시면 그에 맞는 구체적인 문제와 풀이를 바로 만들어 드릴게요. 어떤 주제로 시작할까요?
Human: 철수는 강아지를 세마리 키우고 있습니다.
AI: 좋아요! 그렇다면 강아지와 관련된 문제를 하나 만들어보겠습니다.

**문제:**
철수가 키우는 세 강아지의 몸무게는 각각 5kg, 7kg, 9kg이다. 이 세 강아지의 몸무게를 모두 더한 후, 평균 몸무게를 구하시오.

**풀이:**
세 강아지의 몸무게를 각각 a, b, c로 표시하면